In [57]:
import pandas as pd
import numpy as np
from gtfs_functions import Feed, gtfs_plots
import os
import folium
import networkx as nx
import matplotlib.pyplot as plt
from rich import print as rprint
import seaborn as sns

## Loading data


In [2]:
feed = Feed("./STM GTFS/gtfs_stm.zip", busiest_date=False)

In [5]:
routes = feed.routes
trips = feed.trips
stops = feed.stops
stop_times = feed.stop_times
shapes = feed.shapes
segments = feed.segments

INFO:root:Reading "routes.txt".
INFO:root:accessing trips
INFO:root:Reading "trips.txt".


INFO:root:Reading "stop_times.txt".
INFO:root:_trips is defined in stop_times
INFO:root:Reading "stops.txt".
INFO:root:computing patterns
INFO:root:Reading "shapes.txt".


## Plotting a single trip of a given route


In [ ]:
ROUTE_NAME = "61"

route_id = routes[routes["route_short_name"] == ROUTE_NAME]["route_id"].iloc[0]
trip_ids = trips[trips["route_id"] == route_id]["trip_id"].tolist()
first_trip_id = trip_ids[0]

geom_shape = shapes[
    shapes["shape_id"] == trips[trips["trip_id"] == first_trip_id]["shape_id"].iloc[0]
]

stop_seq = stop_times[stop_times["trip_id"] == first_trip_id].sort_values(
    by="stop_sequence"
)
stop_seq_ids = stop_seq["stop_id"].tolist()
stops_to_plot = stops[stops["stop_id"].isin(stop_seq_ids)]

In [ ]:
m = folium.Map(location=[45.508888, -73.561668], zoom_start=12, tiles="cartodbpositron")

folium.GeoJson(geom_shape["geometry"].iloc[0]).add_to(m)

for index, row in stops_to_plot.iterrows():
    location = (row["stop_lat"], row["stop_lon"])
    folium.CircleMarker(
        location, tooltip=row["stop_name"], radius=2, color="red"
    ).add_to(m)


m

## Getting all stops on a route


In [ ]:
ROUTE_NAME = "35"

route_id = routes[routes["route_short_name"] == ROUTE_NAME]["route_id"].iloc[0]

# get all stops on route 35
stops_on_route = stop_times[
    stop_times["trip_id"].isin(trips[trips["route_id"] == route_id]["trip_id"].tolist())
].drop_duplicates(subset=["stop_id"])

# plot
m = folium.Map(location=[45.508888, -73.561668], zoom_start=12, tiles="cartodbpositron")

for index, row in stops_on_route.iterrows():
    location = (row["stop_lat"], row["stop_lon"])
    folium.CircleMarker(
        location, tooltip=row["stop_name"], radius=2, color="red"
    ).add_to(m)

m

## Getting all routes for a given stop


In [ ]:
stops

,stop_id,stop_code,stop_name,stop_lat,stop_lon,stop_url,location_type,parent_station,wheelchair_boarding,geometry
0,STATION_M118,10118,STATION ANGRIGNON,45.446466,-73.603118,NaN,1,NaN,1,POINT (-73.60312 45.44647)
1,43,10118,Station Angrignon,45.446466,-73.603118,http://www.stm.info/fr/infos/reseaux/metro/ang...,0,STATION_M118,1,POINT (-73.60312 45.44647)
2,43-01,10118,Station Angrignon,45.446319,-73.603835,NaN,2,STATION_M118,1,POINT (-73.60384 45.44632)
3,STATION_M120,10120,STATION MONK,45.451158,-73.593242,NaN,1,NaN,2,POINT (-73.59324 45.45116)
4,42,10120,Station Monk,45.451158,-73.593242,http://www.stm.info/fr/infos/reseaux/metro/monk,0,STATION_M120,2,POINT (-73.59324 45.45116)
...,...,...,...,...,...,...,...,...,...,...
9039,60987,60987,Institut national de santé publique (Sainte-Marie,45.433289,-73.907005,https://www.stm.info/fr/recherche#stq=60987,0,NaN,2,POINT (-73.90700 45.43329)
9040,61121,61121,du Souvenir / du Boulevard (Hôpital Ste-Anne),45.411347,-73.949240,https://www.stm.info/fr/recherche#stq=61121,0,NaN,2,POINT (-73.94924 45.41135)
9041,61253,61253,Lakeshore / Macdonald,45.403753,-73.940191,https://www.stm.info/fr/recherche#stq=61253,0,NaN,1,POINT (-73.94019 45.40375)
9042,61274,61274,SRB Pie-IX / Saint-Martin (zone B),45.610343,-73.662089,https://www.stm.info/fr/recherche#stq=61274,0,NaN,1,POINT (-73.66209 45.61034)


In [ ]:
stop_times[stop_times["stop_code"] == 54305]["route_id"].unique()

array(['359', '45'], dtype=object)

## Number of trips per route


In [ ]:
num_trips = (
    trips.groupby(["route_id"]).nunique()["trip_id"].sort_values(ascending=False)
)

num_trips

route_id
1      3556
10     1742
100    2120
101     130
102    1552
       ... 
94     1811
95     1586
968     796
97     2129
99     1260
Name: trip_id, Length: 220, dtype: int64

In [ ]:
line_frequency = feed.lines_freq

In [ ]:
line_frequency

,route_id,route_name,direction_id,window,min_per_trip,ntrips,geometry,radius,fill_color
0,1,1 Ligne 1 - Verte,0,0:00-6:00,3,93,"LINESTRING (-73.60312 45.44647, -73.59324 45.4...",1.121834,#e895b3ff
1,1,1 Ligne 1 - Verte,0,15:00-19:00,0,444,"LINESTRING (-73.60312 45.44647, -73.59324 45.4...",5.355850,#0e8955ff
2,1,1 Ligne 1 - Verte,0,19:00-22:00,0,254,"LINESTRING (-73.60312 45.44647, -73.59324 45.4...",3.063932,#3ab071ff
3,1,1 Ligne 1 - Verte,0,22:00-24:00,0,141,"LINESTRING (-73.60312 45.44647, -73.59324 45.4...",1.700844,#55d992ff
4,1,1 Ligne 1 - Verte,0,6:00-9:00,0,271,"LINESTRING (-73.60312 45.44647, -73.59324 45.4...",3.268999,#3ab071ff
...,...,...,...,...,...,...,...,...,...
2358,99,99 Villeray,1,15:00-19:00,1,144,"LINESTRING (-73.59286 45.56888, -73.59302 45.5...",1.737033,#55d992ff
2359,99,99 Villeray,1,19:00-22:00,2,80,"LINESTRING (-73.59286 45.56888, -73.59302 45.5...",0.965018,#e895b3ff
2360,99,99 Villeray,1,22:00-24:00,2,50,"LINESTRING (-73.59286 45.56888, -73.59302 45.5...",0.603136,#d13870ff
2361,99,99 Villeray,1,6:00-9:00,2,84,"LINESTRING (-73.59286 45.56888, -73.59302 45.5...",1.013269,#e895b3ff


In [ ]:
most_frequent_lines = (
    (
        line_frequency.groupby(["route_id", "route_name", "direction_id", "geometry"])[
            "ntrips"
        ]
    )
    .sum()
    .reset_index()
    .query("route_id != ['1', '2', '3', '4']")
    .sort_values(by="ntrips", ascending=False)
)

In [ ]:
top_5_lines = most_frequent_lines["route_id"].unique()[:5]
most_frequent_lines

,route_id,route_name,direction_id,geometry,ntrips
121,165,165 Côte-des-Neiges,0,"LINESTRING (-73.57911 45.49565, -73.57922 45.4...",2094
95,141,141 Jean-Talon Est,0,"LINESTRING (-73.59968 45.55985, -73.59966 45.5...",2053
97,141,141 Jean-Talon Est,1,"LINESTRING (-73.53463 45.59652, -73.53481 45.5...",2006
122,165,165 Côte-des-Neiges,1,"LINESTRING (-73.64407 45.51603, -73.64410 45.5...",1992
17,105,105 Sherbrooke,1,"LINESTRING (-73.60432 45.47375, -73.60439 45.4...",1846
...,...,...,...,...,...
156,18,18 Beaubien,0,"LINESTRING (-73.60465 45.53553, -73.60462 45.5...",4
206,197,197 Rosemont,1,"LINESTRING (-73.57379 45.56095, -73.57385 45.5...",4
189,191,191 Broadway / Provost,1,"LINESTRING (-73.57926 45.48313, -73.57917 45.4...",3
65,126,126 Thimens / Grenet,0,"LINESTRING (-73.70172 45.50671, -73.70171 45.5...",3


In [ ]:
line_frequency_plot = line_frequency[line_frequency["route_id"].isin(top_5_lines)]

line_frequency_plot

,route_id,route_name,direction_id,window,min_per_trip,ntrips,geometry,radius,fill_color
81,105,105 Sherbrooke,0,0:00-6:00,4,74,"LINESTRING (-73.64148 45.45453, -73.64160 45.4...",0.892642,#e895b3ff
82,105,105 Sherbrooke,0,15:00-19:00,0,416,"LINESTRING (-73.64148 45.45453, -73.64160 45.4...",5.018094,#0e8955ff
83,105,105 Sherbrooke,0,19:00-22:00,0,185,"LINESTRING (-73.64148 45.45453, -73.64160 45.4...",2.231604,#55d992ff
84,105,105 Sherbrooke,0,22:00-24:00,1,85,"LINESTRING (-73.64148 45.45453, -73.64160 45.4...",1.025332,#e895b3ff
85,105,105 Sherbrooke,0,6:00-9:00,0,254,"LINESTRING (-73.64148 45.45453, -73.64160 45.4...",3.063932,#3ab071ff
...,...,...,...,...,...,...,...,...,...
1965,67,67 Saint-Michel,0,15:00-19:00,0,544,"LINESTRING (-73.55139 45.54693, -73.55144 45.5...",6.562123,#066a40ff
1966,67,67 Saint-Michel,0,19:00-22:00,0,213,"LINESTRING (-73.55139 45.54693, -73.55144 45.5...",2.569361,#3ab071ff
1967,67,67 Saint-Michel,0,22:00-24:00,1,99,"LINESTRING (-73.55139 45.54693, -73.55144 45.5...",1.194210,#e895b3ff
1968,67,67 Saint-Michel,0,6:00-9:00,0,202,"LINESTRING (-73.55139 45.54693, -73.55144 45.5...",2.436671,#3ab071ff
